In [15]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import traitlets # <- to synchronize stuffs
from IPython.display import display
from pandas import DataFrame as DF
import inspect

# A. Widgets Basics
**What and why widgets?**
- Widgets = eventful python objects that have a representation in the browser, often as a control like a slider, textbox, etc.
- can use widgets to build **interactive GUIs** for your notebooks.  
- can also use widgets to **synchronize stateful and stateless information** between Python and JavaScript.

## All the widge types available
- run `widgets.Widget.widget_types.values()` to display all
- **Widget** and **DOMWidget**, not listed below, are base classes.

In [16]:
widgets.Widget.widget_types.values()

[ipywidgets.widgets.widget_string.Text,
 ipywidgets.widgets.widget_string.Latex,
 ipywidgets.widgets.widget_box.Box,
 ipywidgets.widgets.widget_controller.Axis,
 ipywidgets.widgets.widget_bool.Checkbox,
 ipywidgets.widgets.widget_int.IntRangeSlider,
 ipywidgets.widgets.widget_selection.RadioButtons,
 ipywidgets.widgets.widget_string.HTML,
 ipywidgets.widgets.widget_float.FloatRangeSlider,
 ipywidgets.widgets.widget_box.PlaceProxy,
 ipywidgets.widgets.widget_selection.ToggleButtons,
 ipywidgets.widgets.widget_int.IntText,
 ipywidgets.widgets.widget_selection.Dropdown,
 ipywidgets.widgets.widget_bool.Valid,
 ipywidgets.widgets.widget_bool.ToggleButton,
 ipywidgets.widgets.widget_float.FloatSlider,
 ipywidgets.widgets.widget_int.IntProgress,
 ipywidgets.widgets.widget_selection.SelectMultiple,
 ipywidgets.widgets.widget_float.FloatProgress,
 ipywidgets.widgets.widget_color.ColorPicker,
 ipywidgets.widgets.widget_box.FlexBox,
 ipywidgets.widgets.widget_string.Textarea,
 ipywidgets.widgets.

# B. Using interact and interaction

**interact vs interaction**
- interactive returns a Widget instance rather than immediately displaying the widget. (a **box** to be more specific)
    - **box** = a container for other widgets.
- (`interactive` must be used with `IPython.display`, but more flexible)
- when you want a final **return value**, this is super useful (`obj.result` attribute)
- note: i don't see the usecase of using `@interact` (ie as decorator)

```python
w = interactive(lambda x: x, x=(10,15,1)) # <- gives an integer slider
display(w)
w.kwargs
w.result
w.close() 
```

In `interact`| widget abbreviation
---|:---
`x = True` | widgets.Checkbox
`x = 'some string'`| widets.Text
`x = (-10, 10, 2)`| widgets.IntSlider
`x = (-10., 2, 0.1)`| widgets.FloatSlider
`x = ('tuple','of','string')`| widgets.Dropdown

In [17]:
# lambda function that just returns itself
a = interact(lambda x: x, x='str')    # textbox
b = interact(lambda x: x, x=True)     # check-box
c = interact(lambda x: x, x=(-10,10)) # int-slider
d = interact(lambda x: x, x=500)      # int-slider from (-x, 3*x), origin at x
e = interact(lambda x: x, x=(-10,10,0.5)) # (min, max, step) slider
interact(lambda x: x, x=widgets.IntSlider(min=-10,max=30,step=1,value=10)); # <- more explicit method calls
f = interact(lambda x: x, x=('fuck','this','shit')) #dropdown
g = interact(lambda x: x, x={'key1':'val1', 'key2':32}) #dropdown

32

In [18]:
def h(p,q):
    return (p,q)
interact(h,p=5,q=5)
# interact(lambda p,q:(p,q), p=5,q=5) # <- equivalent
# interact(h,p=5,q=fixed(5))

(5, 5)

<function __main__.h>

In [19]:
w = interactive(lambda x: x, x=(10,15,1)) # <- gives an integer slider
display(w)

12

- **box** = a container for other widgets.

## widget abbreviation vs direct-call

In [20]:
w = interactive(lambda x: x, x=(10,15,1))
display(w)

ww = widgets.IntSlider(min=-10,max=30,step=1,value=10)
display(ww)

print(type(interactive(lambda x: x, x=(10,15,1))))  # <- so interactive creates a box
print(type(widgets.IntSlider(min=-10,max=30,step=1,value=10))) # <- explicit call

12

<class 'ipywidgets.widgets.widget_box.Box'>
<class 'ipywidgets.widgets.widget_int.IntSlider'>


In [21]:
# different attribute stores result (result and value...)
print(w.result)

print(ww.value)


12
10


In [22]:
w = widgets.IntSlider(min=-10,max=30,step=1,value=10)
display(ww)
DF(dir(ww)).T

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
0,__class__,__del__,__delattr__,__dict__,__doc__,__format__,__getattribute__,__getstate__,__hash__,__init__,...,traits,unobserve,unobserve_all,update_config,value,version,visible,widget_types,widgets,width


In [23]:
ww = widgets.IntSlider(min=-10,max=30,step=1,value=10)
display(ww)
DF(dir(ww)).T

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
0,__class__,__del__,__delattr__,__dict__,__doc__,__format__,__getattribute__,__getstate__,__hash__,__init__,...,traits,unobserve,unobserve_all,update_config,value,version,visible,widget_types,widgets,width


# C. Widget Events

## (quick summary)
**Relevant functions** (don't foget to run `display(obj)`)

Object instantiation | callback 
------------------|--------------------------
`obj = widgets.Button(description='string')` | `obj.on_click(fhandle)`
`obj = widgets.Text()`   | `obj.on_submit(fhandle)`
`obj=widgets.IntSlider()`| `obj.on_trait_change(fhandle, 'value')`
`caption = widgets.Latex(value = 'str')` | `display(caption, slider1, slider2)`
`l = traitlets.link((sliders1, 'value'), (slider2, 'value'))`| `l.unlink()` (client side)
`dl = traitlets.dlink((source, 'value'), (target1, 'value'))`| `dl.unlink()` (client side)
`l = widgets.jslink((range1, 'value'), (range2, 'value'))` | `l.unlink()` (server side...**probably wont use**)
`dl = widgets.jsdlink((range1, 'value'), (range2, 'value'))` | `dl.unlink()` (server side)

```python
print(widgets.Widget.on_trait_change.__doc__)
```

**4 possible callback Signatures**

Mentioned in the doc string, the callback registered can have 4 possible signatures:

* `callback()`
* `callback(trait_name)`
* `callback(trait_name, new_value)`
* `callback(trait_name, old_value, new_value)`

## widgets.Button

In [24]:
button = widgets.Button(description="Click Me!")
display(button)

# oddly, this requires argument
def on_button_clicked(this_can_be_any_fucking_variable_name):
    print("Button clicked.")

button.on_click(on_button_clicked)
DF(dir(button)).T

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,__class__,__del__,__delattr__,__dict__,__doc__,__format__,__getattribute__,__getstate__,__hash__,__init__,...,trait_names,traits,unobserve,unobserve_all,update_config,version,visible,widget_types,widgets,width


## widgets.Text

In [25]:
text = widgets.Text()
display(text)

def handle_submit(any_var_name):
    print(text.value)

text.on_submit(handle_submit)

## obj.on_trait_change(on_value_change, 'value')

In [26]:
# print(inspect.getargspec(widgets.Widget.on_trait_change))
# print(widgets.Widget.on_trait_change.__doc__)

In [27]:
int_range = widgets.IntSlider()
display(int_range) # <- display will show the display

def on_value_change(name, value):
    print(value)

int_range.on_trait_change(on_value_change, 'value')

C:\Users\takanori\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: on_trait_change is deprecated: use observe instead


## traitlets.link() - (linking/syncing widgets)

In [28]:
caption = widgets.Latex(value = 'The values of slider1 and slider2 are synchronized $\\beta$')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                             widgets.IntSlider(description='Slider 2')
l = traitlets.link((sliders1, 'value'), (slider2, 'value'))
display(caption, sliders1, slider2)

In [29]:
caption = widgets.Latex(value = 'Changes in source_range values are reflected in target_range1')
source_range, target_range1 = widgets.IntSlider(description='Source range'),\
                                             widgets.IntSlider(description='Target range 1')
dl = widgets.jsdlink((source_range, 'value'), (target_range1, 'value'))
display(caption, source_range, target_range1)

# D. Basic styling

The widgets distributed with IPython can be styled by setting the following traits:

- width  
- height  
- background_color  
- border_color  
- border_width  
- border_style  
- font_style  
- font_weight  
- font_size  
- font_family  

The example below shows how a `Button` widget can be styled:

In [30]:
button = widgets.Button(
    description='Hello World!',
    width=100, # Integers are interpreted as pixel measurements.
    height='2em', # em is valid HTML unit of measurement.
    color='lime', # Colors can be set by name,
    background_color='#0022FF', # and also by color code.
    border_color='red')
display(button)